# Prophesee Event Camera 

In this tutorial we demonstrate how to stream event-based data from a Prophesee camera (or recording) into Lava. We show the basic usage of the `PropheseeCamera` Process and how to apply filters and transformations.

In [1]:
import matplotlib
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

from lava.magma.core.run_configs import Loihi2SimCfg
from lava.magma.core.run_conditions import RunSteps

from lava.lib.peripherals.dvs.prophesee import PropheseeCamera

from utils import EventVisualizer

## Download raw event recording

Open the RAW event data file. If the file does not exist, it will be downloaded from Prophesee's public sample server.


In [2]:
import metavision_core as mv

EVENT_RECORDING_FILENAME = "80_balls.raw"
mv.utils.get_sample(EVENT_RECORDING_FILENAME)

reader = mv.event_io.RawReader(EVENT_RECORDING_FILENAME)
height, width = reader.get_size()
del reader

## Basic usage

The `PropheseeCamera` Process can be used to stream from a Prophesee camera or to stream from a recording. To stream from a recording, specify the file name and path in the `filename` parameter. To stream from a camera, do not specify the parameter or set it to an empty string ("").

The output of the `PropheseeCamera` is an event histogram over all pixels, holding the number of events per pixel. The time span over which this histogram is generated can be specified with the optional `num_output_time_bins` parameter, which defaults to 1.


<table>
<tr>
    <td> <img src="gifs/basic.gif" alt="Drawing" style="height: 250px;"/> </td>
</tr>
</table>

In [3]:

# Initialize Processes
camera = PropheseeCamera(filename=EVENT_RECORDING_FILENAME,
                         sensor_shape=(height, width))

event_visualizer = EventVisualizer(shape=camera.s_out.shape)

# Connect
camera.s_out.connect(event_visualizer.s_in)

# Run
num_steps = 200
run_cfg = Loihi2SimCfg()
run_cnd = RunSteps(num_steps=num_steps)

camera.run(condition=run_cnd, run_cfg=run_cfg)
camera.stop()

## Apply filters

The `PropheseeCamera` Process can preprocess event-based data using filters from the metavision_sdk. For instance, the falling balls in the recording cause a trail of events. Such trails often cause undesirable blurring of moving objects in the scene. To reduce such trails, apply the `TrailFilterAlgorithm` from the metavision_sdk. Refer to the metavision documentation for more filters and their usage.


<table>
<tr>
    <td> <img src="gifs/filters.gif" alt="Drawing" style="height: 250px;"/> </td>
</tr>
</table>

In [4]:
from metavision_sdk_cv import TrailFilterAlgorithm, ActivityNoiseFilterAlgorithm


filters = [TrailFilterAlgorithm(width=width, height=height, threshold=100000),
           ActivityNoiseFilterAlgorithm(width=width, height=height, threshold=1000),]


# Initialize Processes
camera = PropheseeCamera(filename=EVENT_RECORDING_FILENAME,
                         filters=filters,
                         sensor_shape=(height, width))

event_visualizer = EventVisualizer(shape=camera.s_out.shape)

# Connect
camera.s_out.connect(event_visualizer.s_in)

# Run
num_steps = 200
run_cfg = Loihi2SimCfg()
run_cnd = RunSteps(num_steps=num_steps)

camera.run(condition=run_cnd, run_cfg=run_cfg)
camera.stop()

## Apply transformations

Lava-peripherals includes transformations that can be applied to the event data, for instance down-sampling, merging of polarities, and mirroring.


<table>
<tr>
    <td> <img src="gifs/transform.gif" alt="Drawing" style="height: 125px;"/> </td>
</tr>
</table>

In [5]:
from lava.lib.peripherals.dvs.transform import Compose, Downsample

transformations = Compose(
    [
        Downsample(factor=0.5),
    ]
)

# Initialize Processes
camera = PropheseeCamera(filename=EVENT_RECORDING_FILENAME,
                              transformations=transformations,
                              sensor_shape=(height, width),
                              num_output_time_bins=1)

event_visualizer = EventVisualizer(shape=camera.s_out.shape)

# Connect
camera.s_out.connect(event_visualizer.s_in)

# Run
num_steps = 200
run_cfg = Loihi2SimCfg()
run_cnd = RunSteps(num_steps=num_steps)

camera.run(condition=run_cnd, run_cfg=run_cfg)
camera.stop()

## Custom transformation and manual output shape

The implementation of transformations is compatible with the transformations from both tonic and torchvision. Please refer to the tonic documentation to see a complete [list of available transformations and their usage](https://tonic.readthedocs.io/en/latest/auto_examples/index.html). If you need to use custom transformation, the output shape of the Process can not be determined automatically. In that case, you need to specify the output shape manually.


<table>
<tr>
    <td> <img src="gifs/custom_trans.gif" alt="Drawing" style="height: 250px;"/> </td>
</tr>
</table>

In [6]:
def expand_x_dim(events):
    events['x'] += 500
    return events
    

transformations = Compose(
    [
        expand_x_dim,
    ]
)

# Initialize Processes
camera = PropheseeCamera(filename=EVENT_RECORDING_FILENAME,
                         transformations=transformations,
                         sensor_shape=(height, width),
                         out_shape=(1, 2, height, width+500))

event_visualizer = EventVisualizer(shape=camera.s_out.shape)

# Connect
camera.s_out.connect(event_visualizer.s_in)

# Run
num_steps = 200
run_cfg = Loihi2SimCfg()
run_cnd = RunSteps(num_steps=num_steps)

camera.run(condition=run_cnd, run_cfg=run_cfg)
camera.stop()